In [ ]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Branch and branch predictions

## Why do we have "branches" in code?

Consider the following code snippet, how does the compiler translate to instructions?

In [ ]:
render_code("loop.c", show="loop0")

In [ ]:
! gcc -S -O0 loop.c
render_code("loop.s", show=["loop0","LFE23"])

In [ ]:
! gcc -S -O2 loop.c
render_code("loop.s", show=["loop1","LFE24"])

In [ ]:
! gcc -S -O2 loop.c
render_code("loop.s", show=["loop2","LFE25"])

## Sorting and branch miss rates

Do you remember this?

In [37]:
compare([do_render_code("arraySort.cpp",show=["//START","//END"]),do_render_code("calculate_sum.c", show="calculate_sum")])

In [38]:
! make clean; make
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "131072,1000,0," >> stats.csv
! cs203 job memory "./arraySort 131072 1000 0"
! echo -n "131072,1000,1," >> stats.csv
! cs203 job memory "./arraySort 131072 1000 1"

rm -f madd arraySort bitonic quicksort arraySortO3
gcc calculate_sum.c -c 
g++ -O0 -DHAVE_LINUX_PERF_EVENT_H arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:46:22: warning: zero-length gnu_printf format string [-Wformat-zero-length]
   46 |     sprintf(preamble,"");
      |                      ^~
srun -N1 -p cs203m ./arraySort 131072 1000 0
sum = 64494148
srun -N1 -p cs203m ./arraySort 131072 1000 1
sum = 64494148


In [39]:
display_df_mono(render_csv("stats.csv"))

,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,131072,1000,0,1506911236,2129827682,1.413373,0.220195,0.468978,0.006985,8227404,1177868618,262260784,35971719
1,131072,1000,1,1615391510,978583361,0.605787,0.262309,0.256691,0.006628,8272997,1248234192,279668925,939287


What did we learn?

The CPI is ??? smaller with data sorted (including sorting itself)

The ET is ??? faster with data sorted (including sorting itself)

What's the cost of branch misses?

Let's exclude the sorting part and do it again.

In [40]:
! make clean; make
! echo "size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "131072,1000,0," >> stats.csv
! cs203 job memory "./arraySort 131072 1000 0"
! echo -n "131072,1000,1," >> stats.csv
! cs203 job memory "./arraySort 131072 1000 1"
display_df_mono(render_csv("stats.csv"))

rm -f madd arraySort bitonic quicksort arraySortO3
make: Warning: File 'arraySort.cpp' has modification time 1.8 s in the future
gcc calculate_sum.c -c 
g++ -O0 -DHAVE_LINUX_PERF_EVENT_H arraySort.cpp perfstats.o calculate_sum.o -o arraySort
arraySort.cpp: In function ‘int main(int, char**)’:
arraySort.cpp:46:22: warning: zero-length gnu_printf format string [-Wformat-zero-length]
   46 |     sprintf(preamble,"");
      |                      ^~
make: warning:  Clock skew detected.  Your build may be incomplete.
srun -N1 -p cs203m ./arraySort 131072 1000 0
sum = 64494148
srun -N1 -p cs203m ./arraySort 131072 1000 1
sum = 64494148


,size,iterations,sorted,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,131072,1000,0,1507073591,2123228174,1.408842,0.266705,0.566276,0.006992,8235604,1177943504,262289522,35759070
1,131072,1000,1,1506636703,894804938,0.593909,0.221225,0.197953,0.006974,8213506,1177735512,262212892,12869


In [ ]:
Diff_Misses = 35759070-5660
Diff_Cycles = 2135192468-898505334
print(Diff_Cycles/Diff_Misses)